# Differential Capacity Analysis with PyBaMM and Transformer
This notebook generates synthetic differential capacity analysis (DCA) curves using [PyBaMM](https://www.pybamm.org/) with randomized parameters. A simple Transformer model is then trained to predict the open circuit potential (OCP) curve from the DCA data.

In [ ]:
# Install compatible package versions
!pip install pybamm==25.6.0 torch==2.3.0 numpy==1.26.4 --quiet


In [ ]:
import pybamm
import torch
import numpy as np
from torch import nn

In [ ]:
def simulate_dca(ocp_scale=1.0, pos_diffusivity=1e-14, particle_radius=1e-6):
    param = pybamm.ParameterValues(
        {
            'Positive electrode diffusivity [m2.s-1]': pos_diffusivity,
            'Positive particle radius [m]': particle_radius,
        }
    )
    model = pybamm.lithium_ion.DFN()
    # Scale the positive electrode OCP with a custom function
    def ocp_mod(c):
        return ocp_scale * pybamm.lithium_ion.stephan_2013.positive_electrode_ocp(c)
    param.update({'Positive electrode OCP [V]': ocp_mod})
    sim = pybamm.Simulation(model, parameter_values=param)
    t_eval = np.linspace(0, 3600, 200)
    sim.solve(t_eval=t_eval)
    Q = sim.solution['Discharge capacity [A.h]'].data
    V = sim.solution['Terminal voltage [V]'].data
    dQdV = np.gradient(Q, V)
    return V, dQdV, ocp_mod


In [ ]:
# Generate synthetic dataset
num_samples = 10000  # modify for a smaller demonstration if needed
seq_len = 200
dca_data = np.zeros((num_samples, seq_len))
ocp_curves = np.zeros((num_samples, seq_len))
voltage_axis = None

for i in range(num_samples):
    ocp_scale = 0.95 + 0.1 * np.random.rand()
    diff = 1e-14 * 10**np.random.uniform(-1, 1)
    radius = 1e-6 * 10**np.random.uniform(-1, 1)
    V, dQdV, ocp_fun = simulate_dca(ocp_scale, diff, radius)
    if voltage_axis is None:
        voltage_axis = V
    dca_data[i] = dQdV
    ocp_curves[i] = ocp_fun(pybamm.Array(V)).entries

np.savez('dca_dataset.npz', X=dca_data, y=ocp_curves, V=voltage_axis)

In [ ]:
# Simple Transformer model for seq2seq OCP prediction
class SeqTransformer(nn.Module):
    def __init__(self, seq_len, d_model=64, nhead=8, num_layers=2):
        super().__init__()
        self.pos = nn.Parameter(torch.randn(seq_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, 1)
    def forward(self, x):
        x = x + self.pos
        h = self.encoder(x)
        out = self.fc(h)
        return out.squeeze(-1)

model = SeqTransformer(seq_len)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Prepare data loaders
X = torch.tensor(dca_data, dtype=torch.float32)
y = torch.tensor(ocp_curves, dtype=torch.float32)
dataset = torch.utils.data.TensorDataset(X, y)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Training loop (short example)
for epoch in range(5):
    for batch_x, batch_y in loader:
        optimizer.zero_grad()
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}:', loss.item())